# Import Libraries

In [1]:
import pandas as pd
from pandas_profiling import ProfileReport
import requests
from math import floor

# Read Data

In [2]:
file='C:/Users/Amit/Desktop/Internships/KPMG/dataset.xlsx'
df4 = pd.read_excel(file, sheet_name="NewCustomerList",header=1)

# Pre-processing

In [3]:
# filter rows
df4=df4.loc[df4['deceased_indicator']=='N']

# replacing values in columns
df4=df4.replace({'gender': {'F': 'Female','M' : 'Male','Femal':'Female'}})
df4=df4.replace({'state': {'New South Wales': 'NSW','Victoria' : 'VIC'}})

# filling missing values in tenure column
df4['tenure'] = df4['tenure'].fillna((floor(df4['tenure'].mean())))

# filling missing values in DOB column
u = df4.select_dtypes(include=['datetime'])
df4[u.columns] = u.fillna(pd.to_datetime('2017'))

# Feature Engineering

In [4]:
# getting age of customer
df4['age']=df4['DOB'].apply(lambda x: 2017- int(str(x)[:4]))
df4['age'].replace(0,39, inplace=True)

# Feature Selection

In [5]:
df4=df4[['gender','age',
         'job_industry_category', 'wealth_segment', 'owns_car', 
         'tenure', 'state', 'property_valuation','past_3_years_bike_related_purchases']]

In [6]:
# removing duplicate rows
df4=df4.drop_duplicates()
df5=df4.copy()

# Encoding Variables

In [7]:
scale_mapper = {'Mass Customer':1, 
                'Affluent Customer':2,
                'High Net Worth':3}
# Map feature values to scale
df5['wealth_segment'] = df5['wealth_segment'].replace(scale_mapper)

In [8]:
x1 = pd.get_dummies(df5['gender'], prefix='gender')
x2 = pd.get_dummies(df5['job_industry_category'], prefix='job')
x3 = pd.get_dummies(df5['owns_car'], prefix='car')
x4 = pd.get_dummies(df5['state'], prefix='state')

In [9]:
X=pd.concat([x1,x2,x3,x4,df5[['age','wealth_segment','tenure','property_valuation']]],axis=1)            
X=X.drop(['gender_U','job_Telecommunications','car_Yes','state_VIC'],axis=1)

In [10]:
X.shape

(1000, 17)

In [11]:
X.to_csv('test.csv',index=False)